# **Import**

In [29]:
import warnings

warnings.filterwarnings('ignore')

In [30]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler

from sklearn.linear_model import Ridge, Lasso, ElasticNet

from sklearn.metrics import mean_absolute_error, mean_squared_error

# **Data Load**

In [31]:
cd /content/drive/MyDrive/[Projects]/Kaggle/NASA 터보팬 제트 엔진의 잔존 수명 예측 및 예지보전 방안/Data

/content/drive/MyDrive/[Projects]/Kaggle/NASA 터보팬 제트 엔진의 잔존 수명 예측 및 예지보전 방안/Data


In [32]:
index_names = ['unit_number', 'time_cycles']
setting_names = ['setting_1', 'setting_2', 'setting_3']
sensor_names = [f's_{i + 1}' for i in range(21)]
col_names = index_names + setting_names + sensor_names

train_df = pd.read_csv('./train_FD001.csv')

In [33]:
max_cycle = train_df.groupby('unit_number')['time_cycles'].max().reset_index()
max_cycle.columns = ['unit_number', 'max_cycle']

train_df = train_df.merge(max_cycle, on='unit_number', how='left')

train_df['RUL'] = train_df['max_cycle'] - train_df['time_cycles']

In [34]:
test_df = pd.read_csv('./test_FD001.csv')
rul_df = pd.read_csv('./RUL_FD001.csv')

In [35]:
results = []

using_sensors = [
    's_2', 's_3', 's_4', 's_7', 's_8', 's_9', 's_11', 's_12',
    's_13', 's_14', 's_15', 's_17', 's_20', 's_21'
]

## Raw Data

In [36]:
# RUL Clipping X
raw_results = []

scaler_dict = {
    'None': None,
    'MinMaxScaler': MinMaxScaler(),
    'StandardScaler': StandardScaler(),
    'RobustScaler': RobustScaler()
}

model_dict = {
    'Ridge': Ridge(),
    'Lasso': Lasso(),
    'ElasticNet': ElasticNet()
}

for scaler_name, scaler in scaler_dict.items():
    for model_name, model in model_dict.items():
        train_x = train_df[using_sensors]
        train_y = train_df['RUL'].values.ravel()
        test_x = test_df.groupby('unit_number').tail(1)[using_sensors]

        if scaler:
            scaler.fit(train_x)

            train_x = scaler.transform(train_x)
            test_x = scaler.transform(test_x)

        model.fit(train_x, train_y)

        pred = model.predict(test_x)
        true = rul_df['RUL'].values.ravel()

        mae = mean_absolute_error(true, pred)
        rmse = mean_squared_error(true, pred)**0.5

        print(f'Model: {model_name}, Scaler: {scaler_name}, MAE: {mae}, RMSE: {rmse}')

        raw_results.append({
        'Data': 'Raw',
        'Model': model_name,
        'Scaler': scaler_name,
        'MAE': mae,
        'RMSE': rmse
        })
results.extend(raw_results)

Model: Ridge, Scaler: None, MAE: 25.55446550765482, RMSE: 31.966354585882446
Model: Lasso, Scaler: None, MAE: 27.705611910388253, RMSE: 34.41729592090012
Model: ElasticNet, Scaler: None, MAE: 28.273298387850954, RMSE: 34.970990292858495
Model: Ridge, Scaler: MinMaxScaler, MAE: 25.52941877634451, RMSE: 31.93453970522252
Model: Lasso, Scaler: MinMaxScaler, MAE: 26.072247959964724, RMSE: 31.823631681369097
Model: ElasticNet, Scaler: MinMaxScaler, MAE: 34.88035993159629, RMSE: 43.78535527129321
Model: Ridge, Scaler: StandardScaler, MAE: 25.540132515420012, RMSE: 31.952237674644834
Model: Lasso, Scaler: StandardScaler, MAE: 25.628434908020644, RMSE: 32.01134867472448
Model: ElasticNet, Scaler: StandardScaler, MAE: 24.8019244188565, RMSE: 30.750887070628103
Model: Ridge, Scaler: RobustScaler, MAE: 25.540068859291836, RMSE: 31.952031005511675
Model: Lasso, Scaler: RobustScaler, MAE: 25.74912289893159, RMSE: 32.07846725801628
Model: ElasticNet, Scaler: RobustScaler, MAE: 25.244384148262746, RM

In [37]:
pd.DataFrame(raw_results).sort_values(by='MAE')

,Data,Model,Scaler,MAE,RMSE
8,Raw,ElasticNet,StandardScaler,24.801924,30.750887
11,Raw,ElasticNet,RobustScaler,25.244384,30.722924
3,Raw,Ridge,MinMaxScaler,25.529419,31.934540
9,Raw,Ridge,RobustScaler,25.540069,31.952031
6,Raw,Ridge,StandardScaler,25.540133,31.952238
0,Raw,Ridge,None,25.554466,31.966355
7,Raw,Lasso,StandardScaler,25.628435,32.011349
10,Raw,Lasso,RobustScaler,25.749123,32.078467
4,Raw,Lasso,MinMaxScaler,26.072248,31.823632
1,Raw,Lasso,None,27.705612,34.417296


In [38]:
# RUL Clipping O
raw_clipping_results = []

scaler_dict = {
    'None': None,
    'MinMaxScaler': MinMaxScaler(),
    'StandardScaler': StandardScaler(),
    'RobustScaler': RobustScaler()
}

model_dict = {
    'Ridge': Ridge(),
    'Lasso': Lasso(),
    'ElasticNet': ElasticNet()
}

for scaler_name, scaler in scaler_dict.items():
    for model_name, model in model_dict.items():
        train_x = train_df[using_sensors]
        train_y = train_df['RUL']
        test_x = test_df.groupby('unit_number').tail(1)[using_sensors]

        train_y.loc[train_y > 125] = 125
        train_y = train_y.values.ravel()

        if scaler:
            scaler.fit(train_x)

            train_x = scaler.transform(train_x)
            test_x = scaler.transform(test_x)

        model.fit(train_x, train_y)

        pred = model.predict(test_x)
        true = rul_df['RUL']
        true.loc[true > 125] = 125
        true = true.values.ravel()

        mae = mean_absolute_error(true, pred)
        rmse = mean_squared_error(true, pred)**0.5

        print(f'Model: {model_name}, Scaler: {scaler_name}, MAE: {mae}, RMSE: {rmse}')

        raw_clipping_results.append({
        'Data': 'Raw (RUL Clipping)',
        'Model': model_name,
        'Scaler': scaler_name,
        'MAE': mae,
        'RMSE': rmse
        })
results.extend(raw_clipping_results)

Model: Ridge, Scaler: None, MAE: 16.60817421253392, RMSE: 20.811860072640243
Model: Lasso, Scaler: None, MAE: 18.346068176190204, RMSE: 22.544121918687367
Model: ElasticNet, Scaler: None, MAE: 18.61296761132814, RMSE: 22.926500783587443
Model: Ridge, Scaler: MinMaxScaler, MAE: 16.603963734069133, RMSE: 20.818854001032232
Model: Lasso, Scaler: MinMaxScaler, MAE: 20.550098420754157, RMSE: 24.071326877399574
Model: ElasticNet, Scaler: MinMaxScaler, MAE: 30.11122934597502, RMSE: 35.67413430159322
Model: Ridge, Scaler: StandardScaler, MAE: 16.608593232278967, RMSE: 20.82464537775906
Model: Lasso, Scaler: StandardScaler, MAE: 17.09062102202773, RMSE: 21.246751064735122
Model: ElasticNet, Scaler: StandardScaler, MAE: 17.349863017854744, RMSE: 21.200879912392733
Model: Ridge, Scaler: RobustScaler, MAE: 16.608642082029874, RMSE: 20.824660655394247
Model: Lasso, Scaler: RobustScaler, MAE: 17.26701374004043, RMSE: 21.41811254810478
Model: ElasticNet, Scaler: RobustScaler, MAE: 17.75063654681008, 

In [39]:
pd.DataFrame(raw_clipping_results).sort_values(by='MAE')

,Data,Model,Scaler,MAE,RMSE
3,Raw (RUL Clipping),Ridge,MinMaxScaler,16.603964,20.818854
0,Raw (RUL Clipping),Ridge,None,16.608174,20.811860
6,Raw (RUL Clipping),Ridge,StandardScaler,16.608593,20.824645
9,Raw (RUL Clipping),Ridge,RobustScaler,16.608642,20.824661
7,Raw (RUL Clipping),Lasso,StandardScaler,17.090621,21.246751
10,Raw (RUL Clipping),Lasso,RobustScaler,17.267014,21.418113
8,Raw (RUL Clipping),ElasticNet,StandardScaler,17.349863,21.200880
11,Raw (RUL Clipping),ElasticNet,RobustScaler,17.750637,21.605295
1,Raw (RUL Clipping),Lasso,None,18.346068,22.544122
2,Raw (RUL Clipping),ElasticNet,None,18.612968,22.926501


## Global Stat

In [40]:
def make_global_stat(df, is_train=True):
    tmp_df = df.copy()
    stat_df = []

    for unit in tmp_df['unit_number'].unique():
        unit_data = tmp_df[tmp_df['unit_number'] == unit]
        features = {'unit_number': unit}

        for sensor in using_sensors:
            features[f'{sensor}_mean'] = unit_data[sensor].mean()
            features[f'{sensor}_std'] = unit_data[sensor].std()
            features[f'{sensor}_min'] = unit_data[sensor].min()
            features[f'{sensor}_max'] = unit_data[sensor].max()
            features[f'{sensor}_last'] = unit_data[sensor].iloc[-1]
            features[f'{sensor}_median'] = unit_data[sensor].median()
            features[f'{sensor}_trend'] = np.polyfit(unit_data['time_cycles'], unit_data[sensor], 1)[0]  # 선형 추세

        if is_train:
            features['RUL'] = unit_data['RUL'].max()

        stat_df.append(features)

    return pd.DataFrame(stat_df)

In [41]:
stat_train_df = make_global_stat(train_df, is_train=True)
stat_test_df = make_global_stat(test_df, is_train=False)

In [42]:
# RUL Clipping X
global_stat_results = []

scaler_dict = {
    'None': None,
    'MinMaxScaler': MinMaxScaler(),
    'StandardScaler': StandardScaler(),
    'RobustScaler': RobustScaler()
}

model_dict = {
    'Ridge': Ridge(),
    'Lasso': Lasso(),
    'ElasticNet': ElasticNet()
}

for scaler_name, scaler in scaler_dict.items():
    for model_name, model in model_dict.items():
        train_x = stat_train_df.drop(columns=['unit_number', 'RUL'])
        train_y = stat_train_df['RUL'].values.ravel()
        test_x = stat_test_df.groupby('unit_number').tail(1).drop(columns='unit_number')

        if scaler:
            scaler.fit(train_x)

            train_x = scaler.transform(train_x)
            test_x = scaler.transform(test_x)

        model.fit(train_x, train_y)

        pred = model.predict(test_x)
        true = rul_df['RUL'].values.ravel()

        mae = mean_absolute_error(true, pred)
        rmse = mean_squared_error(true, pred)**0.5

        print(f'Model: {model_name}, Scaler: {scaler_name}, MAE: {mae}, RMSE: {rmse}')

        global_stat_results.append({
        'Data': 'Global Stat',
        'Model': model_name,
        'Scaler': scaler_name,
        'MAE': mae,
        'RMSE': rmse
        })
results.extend(global_stat_results)

Model: Ridge, Scaler: None, MAE: 50.55, RMSE: 64.50713138870772
Model: Lasso, Scaler: None, MAE: 50.55, RMSE: 64.50713138870772
Model: ElasticNet, Scaler: None, MAE: 50.55, RMSE: 64.50713138870772
Model: Ridge, Scaler: MinMaxScaler, MAE: 50.55, RMSE: 64.50713138870772
Model: Lasso, Scaler: MinMaxScaler, MAE: 50.55, RMSE: 64.50713138870772
Model: ElasticNet, Scaler: MinMaxScaler, MAE: 50.55, RMSE: 64.50713138870772
Model: Ridge, Scaler: StandardScaler, MAE: 50.55, RMSE: 64.50713138870772
Model: Lasso, Scaler: StandardScaler, MAE: 50.55, RMSE: 64.50713138870772
Model: ElasticNet, Scaler: StandardScaler, MAE: 50.55, RMSE: 64.50713138870772
Model: Ridge, Scaler: RobustScaler, MAE: 50.55, RMSE: 64.50713138870772
Model: Lasso, Scaler: RobustScaler, MAE: 50.55, RMSE: 64.50713138870772
Model: ElasticNet, Scaler: RobustScaler, MAE: 50.55, RMSE: 64.50713138870772


In [43]:
pd.DataFrame(global_stat_results).sort_values(by='MAE')

,Data,Model,Scaler,MAE,RMSE
0,Global Stat,Ridge,None,50.55,64.507131
1,Global Stat,Lasso,None,50.55,64.507131
2,Global Stat,ElasticNet,None,50.55,64.507131
3,Global Stat,Ridge,MinMaxScaler,50.55,64.507131
4,Global Stat,Lasso,MinMaxScaler,50.55,64.507131
5,Global Stat,ElasticNet,MinMaxScaler,50.55,64.507131
6,Global Stat,Ridge,StandardScaler,50.55,64.507131
7,Global Stat,Lasso,StandardScaler,50.55,64.507131
8,Global Stat,ElasticNet,StandardScaler,50.55,64.507131
9,Global Stat,Ridge,RobustScaler,50.55,64.507131


In [44]:
# RUL Clipping O
global_stat_rul_clipping_results = []

scaler_dict = {
    'None': None,
    'MinMaxScaler': MinMaxScaler(),
    'StandardScaler': StandardScaler(),
    'RobustScaler': RobustScaler()
}

model_dict = {
    'Ridge': Ridge(),
    'Lasso': Lasso(),
    'ElasticNet': ElasticNet()
}

for scaler_name, scaler in scaler_dict.items():
    for model_name, model in model_dict.items():
        train_x = stat_train_df.drop(columns=['unit_number', 'RUL'])
        train_y = stat_train_df['RUL']
        test_x = stat_test_df.groupby('unit_number').tail(1).drop(columns='unit_number')

        train_y.loc[train_y > 125] = 125
        train_y = train_y.values.ravel()

        if scaler:
            scaler.fit(train_x)

            train_x = scaler.transform(train_x)
            test_x = scaler.transform(test_x)

        model.fit(train_x, train_y)

        pred = model.predict(test_x)
        true = rul_df['RUL']
        true.loc[true > 125] = 125
        true = true.values.ravel()

        mae = mean_absolute_error(true, pred)
        rmse = mean_squared_error(true, pred)**0.5

        print(f'Model: {model_name}, Scaler: {scaler_name}, MAE: {mae}, RMSE: {rmse}')

        global_stat_rul_clipping_results.append({
        'Data': 'Global Stat (RUL Clipping)',
        'Model': model_name,
        'Scaler': scaler_name,
        'MAE': mae,
        'RMSE': rmse
        })
results.extend(global_stat_rul_clipping_results)

Model: Ridge, Scaler: None, MAE: 50.55, RMSE: 64.50713138870772
Model: Lasso, Scaler: None, MAE: 50.55, RMSE: 64.50713138870772
Model: ElasticNet, Scaler: None, MAE: 50.55, RMSE: 64.50713138870772
Model: Ridge, Scaler: MinMaxScaler, MAE: 50.55, RMSE: 64.50713138870772
Model: Lasso, Scaler: MinMaxScaler, MAE: 50.55, RMSE: 64.50713138870772
Model: ElasticNet, Scaler: MinMaxScaler, MAE: 50.55, RMSE: 64.50713138870772
Model: Ridge, Scaler: StandardScaler, MAE: 50.55, RMSE: 64.50713138870772
Model: Lasso, Scaler: StandardScaler, MAE: 50.55, RMSE: 64.50713138870772
Model: ElasticNet, Scaler: StandardScaler, MAE: 50.55, RMSE: 64.50713138870772
Model: Ridge, Scaler: RobustScaler, MAE: 50.55, RMSE: 64.50713138870772
Model: Lasso, Scaler: RobustScaler, MAE: 50.55, RMSE: 64.50713138870772
Model: ElasticNet, Scaler: RobustScaler, MAE: 50.55, RMSE: 64.50713138870772


In [45]:
pd.DataFrame(global_stat_rul_clipping_results).sort_values(by='MAE')

,Data,Model,Scaler,MAE,RMSE
0,Global Stat (RUL Clipping),Ridge,None,50.55,64.507131
1,Global Stat (RUL Clipping),Lasso,None,50.55,64.507131
2,Global Stat (RUL Clipping),ElasticNet,None,50.55,64.507131
3,Global Stat (RUL Clipping),Ridge,MinMaxScaler,50.55,64.507131
4,Global Stat (RUL Clipping),Lasso,MinMaxScaler,50.55,64.507131
5,Global Stat (RUL Clipping),ElasticNet,MinMaxScaler,50.55,64.507131
6,Global Stat (RUL Clipping),Ridge,StandardScaler,50.55,64.507131
7,Global Stat (RUL Clipping),Lasso,StandardScaler,50.55,64.507131
8,Global Stat (RUL Clipping),ElasticNet,StandardScaler,50.55,64.507131
9,Global Stat (RUL Clipping),Ridge,RobustScaler,50.55,64.507131


## **Cumulative Stat**

In [46]:
def make_cum_stat(df, is_train=True):
    tmp_df = df.copy()
    stat_df = []

    for unit in tmp_df['unit_number'].unique():
        unit_data = tmp_df[tmp_df['unit_number'] == unit]
        for t in range(2, len(unit_data) + 1):  # 최소 2개 시점부터
            sub_data = unit_data.iloc[:t]  # 1~t
            features = {'unit_number': unit, 'time_cycles': sub_data['time_cycles'].iloc[-1]}

            for sensor in using_sensors:
                features[f'{sensor}_mean'] = sub_data[sensor].mean()
                features[f'{sensor}_std'] = sub_data[sensor].std()
                features[f'{sensor}_min'] = sub_data[sensor].min()
                features[f'{sensor}_max'] = sub_data[sensor].max()
                features[f'{sensor}_last'] = sub_data[sensor].iloc[-1]
                features[f'{sensor}_median'] = sub_data[sensor].median()
                # trend는 최소 2개 이상 시점에서만 계산 가능
                features[f'{sensor}_trend'] = np.polyfit(sub_data['time_cycles'], sub_data[sensor], 1)[0]

            if is_train:
                features['RUL'] = sub_data['RUL'].iloc[-1]

            stat_df.append(features)

    return pd.DataFrame(stat_df)

In [47]:
cum_stat_train_df = make_cum_stat(train_df, is_train=True)
cum_stat_test_df = make_cum_stat(test_df, is_train=False)

In [48]:
# RUL Clipping X
cum_stat_results = []

scaler_dict = {
    'None': None,
    'MinMaxScaler': MinMaxScaler(),
    'StandardScaler': StandardScaler(),
    'RobustScaler': RobustScaler()
}

model_dict = {
    'Ridge': Ridge(),
    'Lasso': Lasso(),
    'ElasticNet': ElasticNet()
}

for scaler_name, scaler in scaler_dict.items():
    for model_name, model in model_dict.items():
        train_x = cum_stat_train_df.drop(columns=['unit_number', 'RUL'])
        train_y = cum_stat_train_df['RUL'].values.ravel()
        test_x = cum_stat_test_df.groupby('unit_number').tail(1).drop(columns='unit_number')

        if scaler:
            scaler.fit(train_x)

            train_x = scaler.transform(train_x)
            test_x = scaler.transform(test_x)

        model.fit(train_x, train_y)

        pred = model.predict(test_x)
        true = rul_df['RUL'].values.ravel()

        mae = mean_absolute_error(true, pred)
        rmse = mean_squared_error(true, pred)**0.5

        print(f'Model: {model_name}, Scaler: {scaler_name}, MAE: {mae}, RMSE: {rmse}')

        cum_stat_results.append({
        'Data': 'Cumulative Stat',
        'Model': model_name,
        'Scaler': scaler_name,
        'MAE': mae,
        'RMSE': rmse
        })
results.extend(cum_stat_results)

Model: Ridge, Scaler: None, MAE: 15.16610954721691, RMSE: 18.43667909974664
Model: Lasso, Scaler: None, MAE: 16.5903089362812, RMSE: 20.056766883108196
Model: ElasticNet, Scaler: None, MAE: 16.462405734186074, RMSE: 20.12955396025475
Model: Ridge, Scaler: MinMaxScaler, MAE: 15.331095096471472, RMSE: 18.589269466553002
Model: Lasso, Scaler: MinMaxScaler, MAE: 18.695661131363906, RMSE: 22.256052053413807
Model: ElasticNet, Scaler: MinMaxScaler, MAE: 26.23671397016859, RMSE: 30.30925937883296
Model: Ridge, Scaler: StandardScaler, MAE: 15.64940605474834, RMSE: 18.844000721914078
Model: Lasso, Scaler: StandardScaler, MAE: 14.743519423904736, RMSE: 17.918505628965935
Model: ElasticNet, Scaler: StandardScaler, MAE: 14.95037189385401, RMSE: 17.850735091154423
Model: Ridge, Scaler: RobustScaler, MAE: 15.630869124726782, RMSE: 18.83160239559931
Model: Lasso, Scaler: RobustScaler, MAE: 15.02783886996059, RMSE: 18.076195773675504
Model: ElasticNet, Scaler: RobustScaler, MAE: 15.134981128567917, RM

In [49]:
pd.DataFrame(cum_stat_results).sort_values(by='MAE')

,Data,Model,Scaler,MAE,RMSE
7,Cumulative Stat,Lasso,StandardScaler,14.743519,17.918506
8,Cumulative Stat,ElasticNet,StandardScaler,14.950372,17.850735
10,Cumulative Stat,Lasso,RobustScaler,15.027839,18.076196
11,Cumulative Stat,ElasticNet,RobustScaler,15.134981,17.971473
0,Cumulative Stat,Ridge,None,15.166110,18.436679
3,Cumulative Stat,Ridge,MinMaxScaler,15.331095,18.589269
9,Cumulative Stat,Ridge,RobustScaler,15.630869,18.831602
6,Cumulative Stat,Ridge,StandardScaler,15.649406,18.844001
2,Cumulative Stat,ElasticNet,None,16.462406,20.129554
1,Cumulative Stat,Lasso,None,16.590309,20.056767


In [50]:
# RUL Clipping O
cum_stat_rul_clipping_results = []

scaler_dict = {
    'None': None,
    'MinMaxScaler': MinMaxScaler(),
    'StandardScaler': StandardScaler(),
    'RobustScaler': RobustScaler()
}

model_dict = {
    'Ridge': Ridge(),
    'Lasso': Lasso(),
    'ElasticNet': ElasticNet()
}

for scaler_name, scaler in scaler_dict.items():
    for model_name, model in model_dict.items():
        train_x = cum_stat_train_df.drop(columns=['unit_number', 'RUL'])
        train_y = cum_stat_train_df['RUL']
        test_x = cum_stat_test_df.groupby('unit_number').tail(1).drop(columns='unit_number')

        train_y.loc[train_y > 125] = 125
        train_y = train_y.values.ravel()

        if scaler:
            scaler.fit(train_x)

            train_x = scaler.transform(train_x)
            test_x = scaler.transform(test_x)

        model.fit(train_x, train_y)

        pred = model.predict(test_x)
        true = rul_df['RUL']
        true.loc[true > 125] = 125
        true = true.values.ravel()

        mae = mean_absolute_error(true, pred)
        rmse = mean_squared_error(true, pred)**0.5

        print(f'Model: {model_name}, Scaler: {scaler_name}, MAE: {mae}, RMSE: {rmse}')

        cum_stat_rul_clipping_results.append({
        'Data': 'Cumulative Stat (RUL Clipping)',
        'Model': model_name,
        'Scaler': scaler_name,
        'MAE': mae,
        'RMSE': rmse
        })
results.extend(cum_stat_rul_clipping_results)

Model: Ridge, Scaler: None, MAE: 15.16610954721691, RMSE: 18.43667909974664
Model: Lasso, Scaler: None, MAE: 16.5903089362812, RMSE: 20.056766883108196
Model: ElasticNet, Scaler: None, MAE: 16.462405734186074, RMSE: 20.12955396025475
Model: Ridge, Scaler: MinMaxScaler, MAE: 15.331095096471472, RMSE: 18.589269466553002
Model: Lasso, Scaler: MinMaxScaler, MAE: 18.695661131363906, RMSE: 22.256052053413807
Model: ElasticNet, Scaler: MinMaxScaler, MAE: 26.23671397016859, RMSE: 30.30925937883296
Model: Ridge, Scaler: StandardScaler, MAE: 15.64940605474834, RMSE: 18.844000721914078
Model: Lasso, Scaler: StandardScaler, MAE: 14.743519423904736, RMSE: 17.918505628965935
Model: ElasticNet, Scaler: StandardScaler, MAE: 14.95037189385401, RMSE: 17.850735091154423
Model: Ridge, Scaler: RobustScaler, MAE: 15.630869124726782, RMSE: 18.83160239559931
Model: Lasso, Scaler: RobustScaler, MAE: 15.02783886996059, RMSE: 18.076195773675504
Model: ElasticNet, Scaler: RobustScaler, MAE: 15.134981128567917, RM

In [51]:
pd.DataFrame(cum_stat_rul_clipping_results).sort_values(by='MAE')

,Data,Model,Scaler,MAE,RMSE
7,Cumulative Stat (RUL Clipping),Lasso,StandardScaler,14.743519,17.918506
8,Cumulative Stat (RUL Clipping),ElasticNet,StandardScaler,14.950372,17.850735
10,Cumulative Stat (RUL Clipping),Lasso,RobustScaler,15.027839,18.076196
11,Cumulative Stat (RUL Clipping),ElasticNet,RobustScaler,15.134981,17.971473
0,Cumulative Stat (RUL Clipping),Ridge,None,15.166110,18.436679
3,Cumulative Stat (RUL Clipping),Ridge,MinMaxScaler,15.331095,18.589269
9,Cumulative Stat (RUL Clipping),Ridge,RobustScaler,15.630869,18.831602
6,Cumulative Stat (RUL Clipping),Ridge,StandardScaler,15.649406,18.844001
2,Cumulative Stat (RUL Clipping),ElasticNet,None,16.462406,20.129554
1,Cumulative Stat (RUL Clipping),Lasso,None,16.590309,20.056767


# **Result**

In [52]:
result = pd.DataFrame(results)
result.to_csv('/content/drive/MyDrive/[Projects]/Kaggle/NASA 터보팬 제트 엔진의 잔존 수명 예측 및 예지보전 방안/Results/Regularized_Regression.csv')

In [53]:
result = result.sort_values(by='MAE')
result.head()

,Data,Model,Scaler,MAE,RMSE
55,Cumulative Stat,Lasso,StandardScaler,14.743519,17.918506
67,Cumulative Stat (RUL Clipping),Lasso,StandardScaler,14.743519,17.918506
68,Cumulative Stat (RUL Clipping),ElasticNet,StandardScaler,14.950372,17.850735
56,Cumulative Stat,ElasticNet,StandardScaler,14.950372,17.850735
58,Cumulative Stat,Lasso,RobustScaler,15.027839,18.076196
